<a href="https://colab.research.google.com/github/joheras/MusicaCatedralStoDomingoIER/blob/main/Notebooks/SacarM%C3%BAsica/Creando_Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Datasets para entrenamiento
- Vamos a crear con algunos de los pentagramas un dataset para la posterior clasificación.

## Librerías

In [1]:
%%capture
pip install py_midicsv

In [2]:
%%capture
pip install MIDIUtil

In [3]:
%%capture
pip install pretty_midi

In [4]:
import cv2
import pickle
import midiutil
import pretty_midi
import IPython.display as ipd
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import patches
from matplotlib import colors
import statistics
import py_midicsv as pm
import random
import sys
import json
import os
import re
import music21 as m21

In [5]:
%%capture
pip install icevision[all]

In [6]:
from icevision.all import *

## Funciones

In [7]:
def verPagina(num_obra):
  for cla in ini_Obras:
    if num_obra in ini_Obras[cla]: return cla

In [8]:
def borrarTexto(foto,numPag):
  n = len(texto[numPag]['text'])
  foto2 = foto.copy()
  for i in range(0,n):
    x = texto[numPag]['left'][i]
    y = texto[numPag]['top'][i]
    h = texto[numPag]['height'][i]
    w = texto[numPag]['width'][i]
    cv2.rectangle(foto2, (x-150, y-2), (x + w + 150, y + h+2), (255, 255, 255), -1)
  return foto2

In [9]:
def abrirImagen(path):
  return cv2.imread(str(path))

def imagenDeImagen(img,h=16,w=16):
  plt.figure(figsize = (h,w))
  plt.imshow(img)
  plt.show()

def mostrarImagen(path):
  img=cv2.imread(str(path))
  imagenDeImagen(img)
  return abrirImagen(img)


def imgAnotada(path):
  colores = {'claved':(119, 90, 17), 'claves':(119, 90, 17), 'clavef':(119, 90, 17), 'nota1':(150,255,209), 'nota2':(150,255,209), 'nota4':(150,255,209), 'nota1:2':(150,255,209), 'nota1:4':(150,255,209), 'notae':(150,255,209), 'sos':(242,170,255), 'bem':(242,170,255), 'bec':(242,170,255), 'sil0':(159,142,243), 'sil1':(159,142,243), 'sil2':(159,142,243), 'sil1:2':(159,142,243), 'sil1:4':(159,142,243), '2x4':(173,76,0), '3x4':(173,76,0), '4x4':(173,76,0), '6x8':(173,76,0), 'punt':(0,255,0), 'liga':(40,47,153),'3':(255,0,0),'line':(119,189,255),'other':(35,99,33)}
  img = cv2.imread(str(path))
  n=path.stem+'.json'
  anotacion= json.loads(open(str(path.parent.parent/'annotations'/n)).read())
  for elemento in anotacion:
    x, y, z, w=elemento["box"]
    cv2.rectangle(img, (x, y), (z,w), colores[elemento["label"]], 2)
  return img

def imgEscrita(path):
  colores = {'claved':(119, 90, 17), 'claves':(119, 90, 17), 'clavef':(119, 90, 17), 'nota1':(150,255,209), 'nota2':(150,255,209), 'nota4':(150,255,209), 'nota1:2':(150,255,209), 'nota1:4':(150,255,209), 'notae':(150,255,209), 'sos':(242,170,255), 'bem':(242,170,255), 'bec':(242,170,255), 'sil0':(159,142,243), 'sil1':(159,142,243), 'sil2':(159,142,243), 'sil1:2':(159,142,243), 'sil1:4':(159,142,243), '2x4':(173,76,0), '3x4':(173,76,0), '4x4':(173,76,0), '6x8':(173,76,0), 'punt':(0,255,0), 'liga':(40,47,153),'3':(255,0,0),'line':(119,189,255),'other':(35,99,33)}
  img = cv2.imread(str(path))
  img = imgAnotada(path)
  n=path.stem+'.json'
  anotacion= json.loads(open(str(path.parent.parent/'annotations'/n)).read())
  for elemento in anotacion:
    x, y, z, w=elemento["box"]
    cv2.putText(img, elemento["label"], (int(x+(z-x)/2), w+10), cv2.FONT_HERSHEY_TRIPLEX,0.35, colores[elemento["label"]], 1)
  return img

def pintarPredicciones(num):
  for f in files[0:num]:
    clone=imgAnotada(f)
    txt_pred=open('input/detection-results/'+f.stem+'.txt')
    linea_pred=txt_pred.readline() 
    while linea_pred!='':
      logit,conf,x1,y1,z1,w1=linea_pred.split()
      linea_pred=txt_pred.readline()
      cv2.rectangle(clone, (int(x1), int(y1)), (int(z1),int(w1)), (255, 0, 0), 2)
    imagenDeImagen(clone,31,10)

## Recuperando las fotos

In [10]:
%%capture
!wget https://www.dropbox.com/s/u2ekoltvu35kfjw/Fotos.zip?dl=1 -O Fotos.zip

In [11]:
%%capture
!unzip Fotos.zip

In [12]:
with open("texto.pickle", "rb") as f:
    texto = pickle.load(f)
with open("pentagramas2.pickle", "rb") as f:
    pentagramas2 = pickle.load(f)
with open("ini_Obras.pickle", "rb") as f:
    ini_Obras = pickle.load(f)
with open("Obras_final.pickle", "rb") as f:
    Obras_final = pickle.load(f)
with open("Obras.pickle", "rb") as f:
    Obras = pickle.load(f)

In [13]:
claves = [cla for cla in pentagramas2]
Pagina = {}
for cla in claves:
  foto = cv2.imread('Fotos/pag'+str(cla)+'.jpg')
  foto = cv2.cvtColor(foto,cv2.COLOR_BGR2RGB)
  Pagina[cla] = foto

### Cogiendo una muestra de los pentagramas para entrenar.
 - De un pentagrama salen bastantes símbolos así que de momento no vamos a coger demasiados.
 - Ponemos una semilla (random_state) para obtener siempre la misma muestra

In [14]:
S = 0
for cla in claves:
  S = S + len(pentagramas2[cla])
S

2308

In [15]:
from sklearn.model_selection import train_test_split

In [21]:
k = 1
for cla in claves:
  pag = Pagina[cla]
  for i in range(0,len(pentagramas2[cla])):
    x,y,w,h = pentagramas2[cla][i]
    pent = borrarTexto(pag,cla)[y:y+h,x:x+w]
    cv2.imwrite('/content/pentagramas/pent' + str(k) + '.jpg',pent)
    k = k + 1

In [22]:
muestra = [i for i in range(1,k)]
_, muestra, _, _, = train_test_split(muestra, muestra, test_size=0.13, random_state=42)

In [23]:
len(muestra)

301

In [24]:
print(muestra)

[1643, 510, 1135, 45, 974, 1118, 1746, 1661, 2021, 1248, 1576, 472, 1412, 209, 1242, 508, 1350, 283, 1226, 483, 1407, 941, 1961, 621, 930, 1331, 434, 1056, 1954, 1353, 1930, 2168, 1211, 1104, 801, 1819, 678, 30, 1422, 1863, 2060, 1921, 734, 1042, 1740, 2131, 2277, 1488, 433, 1057, 1343, 2163, 462, 1866, 1765, 1933, 963, 2164, 2286, 773, 1660, 2272, 1533, 1675, 112, 1293, 252, 916, 230, 600, 291, 423, 2187, 1534, 1180, 2072, 975, 135, 1035, 611, 2276, 197, 322, 1977, 479, 1721, 219, 101, 2002, 1291, 424, 57, 1217, 1092, 591, 1468, 1832, 1122, 1727, 743, 1547, 1302, 415, 2178, 1950, 1387, 1026, 2234, 747, 1171, 917, 248, 2177, 2257, 1001, 486, 97, 2120, 180, 1111, 1799, 1011, 1323, 882, 255, 1546, 1780, 446, 436, 760, 1126, 1674, 1205, 631, 1431, 473, 1522, 880, 1772, 565, 529, 890, 1654, 280, 308, 1897, 1893, 2095, 2005, 2150, 1784, 1512, 1434, 799, 509, 409, 950, 1402, 71, 1865, 189, 439, 304, 1718, 1212, 1279, 1884, 260, 1447, 1754, 1191, 1653, 811, 158, 835, 530, 1745, 1335, 421, 160

## Anotando las imágenes
- Nos ayudaremos de OpenCV para anotarlas. La anotación la vamos a hacer "semi-manual". Esto es, sacaremos por pantalla el bounding box de todos los contornos para apuntarlos manualmanete en un archivo.

In [25]:
%%capture
!unzip Musica.zip

In [74]:
muestra[111] ## Toca la 111

917

In [76]:
pent = cv2.imread('pentagramas/pent'+str(muestra[111])+'.jpg')
pent = cv2.cvtColor(pent,cv2.COLOR_BGR2RGB)
cv2.imwrite('/content/Musica/pents/images/pent'+str(muestra[111])+'.jpg',pent)

True

In [ ]:
pent = cv2.imread('pentagramas/pent'+str(muestra[111])+'.jpg')
pent = cv2.cvtColor(pent,cv2.COLOR_BGR2RGB)
p = pent.copy()
p  = cv2.cvtColor(p,cv2.COLOR_RGB2GRAY)
_,p=cv2.threshold(p, 0, 1, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)

# Operaciones con OpenCV

kernel = np.ones((1,100),dtype='uint8')
p = cv2.morphologyEx(p,cv2.MORPH_TOPHAT,kernel)
kernel = np.ones((2,1),dtype='uint8')
p = cv2.erode(p,kernel)
kernel = np.ones((5,2),dtype='uint8')
p = cv2.morphologyEx(p,cv2.MORPH_CLOSE,kernel)
kernel = np.ones((2,4),dtype='uint8')
p = cv2.dilate(p,kernel)
cnts,_ = cv2.findContours(p, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
for c in cnts:
  (x, y, w, h) = cv2.boundingRect(c)
  if w > 40:
    kernel = np.ones((8,1),dtype='uint8')
    p[y-7:y+h+7,x-7:x+w+7] = cv2.morphologyEx(p[y-7:y+h+7,x-7:x+w+7],cv2.MORPH_ELLIPSE,kernel)

# FIN

fig, ax = plt.subplots(1,1,figsize=(16,16))
ax.imshow(p,cmap=plt.cm.binary)
ax.xaxis.set_ticklabels([])
ax.yaxis.set_ticklabels([])
plt.show()
cnts,_ = cv2.findContours(p, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
for c in cnts:
  p = pent.copy()
  (x, y, w, h) = cv2.boundingRect(c)
  cv2.rectangle(p, (x-4, y-4), (x + w+4, y + h+4), (119, 90, 17), 2)
  fig, ax = plt.subplots(1,1,figsize=(16,16))
  ax.imshow(p)
  ax.xaxis.set_ticklabels([])
  ax.yaxis.set_ticklabels([])
  print('{'+'"box": [{:}, {:}, {:}, {:}]'.format(x-4, y-4, x + w+4, y + h+4)+',"label": "nota"},')
  plt.show()

## Anotando notas

In [144]:
import shutil
shutil.make_archive('Musica', 'zip', '/content/Musica')

'/content/Musica.zip'

In [ ]:
%%capture
!unzip Musica.zip

In [78]:
files = get_files('/content/Musica/pents', folders = ['images'])

In [137]:
files.index(Path('/content/Musica/pents/images/pent'+str(muestra[110])+'.jpg')) # 111

104

In [142]:
muestra[110] # 111

917

In [ ]:
img = cv2.imread(str(files[104])) 
n=files[104].stem+'.json'
anotacion= json.loads(open(str(files[104].parent.parent/'annotations'/n)).read())
L = []
k = 0
for elemento in anotacion:
  if 'nota' in elemento['label']:
    xmin,ymin,xmax,ymax = elemento['box']
    fig, ax = plt.subplots(1,1,figsize=(2,2))
    ax.imshow(img[:,xmin-1:xmax+1])
    #ax.set_title(L[k])
    #cv2.imwrite('/content/Musica/notas/'+str(L[k])+'/nota248-'+str(k)+'-'+'.jpg',img[:,xmin-1:xmax+1])  
    k = k + 1